In [1]:
import os

# Why Torch? You'll find the answer in the .md files! 
os.environ["KERAS_BACKEND"] = "torch"

In [2]:
import keras
from torch.utils.data import DataLoader

from dataset.dataset_loader import dataset_loader
import keras_tuner

from utils.my_tuner import HistoryDeletingBayesianOptimization

from dataset.k_fold_dataset_wrapper import KFoldDatasetWrapper

In [3]:
import torch

torch.cuda.is_available()

True

In [4]:
# Initial steps
hyperparameters = keras_tuner.HyperParameters()
train, test = dataset_loader((224, 224), is_grayscale=False)
dataset_split_controller = KFoldDatasetWrapper(5)
dataset_split_controller.load_data(train)

local_train, validation = dataset_split_controller.get_data_for_fold(0)
train_dataloader = DataLoader(dataset=local_train, batch_size=32, shuffle=True)
validation_dataloader = DataLoader(dataset=validation, batch_size=32, shuffle=True)

# First custom attempt
This is done before fine tuning following general rules of thumb for the construction of CNN, which are: 
- The number of filters may be mixed increasingly to better match more complex patterns in the images
- A (3,3) Kernel is generally good
- CNN benefit from having multiple layers

In [28]:
from models.structure.base_model_family import ConvLayerStructure, PoolLayerStructure, HiddenLayerStructure
from models.simple_cnn.conv_net_family import TunableConvNetFamily

custom_conv_family = TunableConvNetFamily()
custom_conv_family.convolution_layers = [
    (ConvLayerStructure(kernel_size=(3, 3), filters=64), PoolLayerStructure((2, 2), 2)),
    (ConvLayerStructure(kernel_size=(3, 3), filters=128), PoolLayerStructure((2, 2), 2))
]

custom_conv_family.dense_layers = [HiddenLayerStructure(64, False)]

model = custom_conv_family.make_model((3, 224, 224))
custom_conv_family.compile_model(model)

history = model.fit(x=train_dataloader, validation_data=validation_dataloader, batch_size=32, epochs=120,
                    callbacks=[
                        # To avoid going further when training
                        keras.callbacks.EarlyStopping(
                            monitor='val_loss', min_delta=1e-4, patience=10,
                            verbose=1, mode='min', restore_best_weights=True
                        ),
                        # To persist the history
                        keras.callbacks.CSVLogger(f"hand-tailored-cnn.log", separator=",", append=True)
                    ])

Epoch 1/120


/home/jacopo/miniconda3/envs/keras-pytorch/lib/python3.12/site-packages/keras/src/backend/torch/nn.py:412: UserWarning:

Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1704987296916/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)



119/119 ━━━━━━━━━━━━━━━━━━━━ 51s 427ms/step - accuracy: 0.5619 - loss: 0.6810 - val_accuracy: 0.6188 - val_loss: 0.6632
Epoch 2/120
119/119 ━━━━━━━━━━━━━━━━━━━━ 48s 405ms/step - accuracy: 0.6364 - loss: 0.6530 - val_accuracy: 0.6515 - val_loss: 0.6321
Epoch 3/120
119/119 ━━━━━━━━━━━━━━━━━━━━ 48s 402ms/step - accuracy: 0.6783 - loss: 0.6162 - val_accuracy: 0.6241 - val_loss: 0.6258
Epoch 4/120
119/119 ━━━━━━━━━━━━━━━━━━━━ 48s 407ms/step - accuracy: 0.7181 - loss: 0.5831 - val_accuracy: 0.6579 - val_loss: 0.6000
Epoch 5/120
119/119 ━━━━━━━━━━━━━━━━━━━━ 47s 397ms/step - accuracy: 0.7146 - loss: 0.5714 - val_accuracy: 0.7191 - val_loss: 0.5528
Epoch 6/120
119/119 ━━━━━━━━━━━━━━━━━━━━ 47s 398ms/step - accuracy: 0.7497 - loss: 0.5399 - val_accuracy: 0.7339 - val_loss: 0.5306
Epoch 7/120
119/119 ━━━━━━━━━━━━━━━━━━━━ 47s 398ms/step - accuracy: 0.7532 - loss: 0.5238 - val_accuracy: 0.7455 - val_loss: 0.5170
Epoch 8/120
119/119 ━━━━━━━━━━━━━━━━━━━━ 48s 399ms/step - accuracy: 0.7694 - loss: 0.492

KeyboardInterrupt: 

# HyperTuning and search
Ho fissato il numero di layer ma vorrei renderlo libero

In [5]:
from models.simple_cnn.conv_net_family import TunableConvNetFamily
from models.structure.tunable_model_family_hypermodel import TunableModelFamilyHypermodel

# For now the optimizer is also fixed to SGD with these parameters:
hyperparameters.Fixed("lr", 1e-4)
hyperparameters.Fixed("momentum", 0.9)

# For now dropout layers are frozen to be disabled.
hyperparameters.Fixed("hidden_layers", 1)
hyperparameters.Fixed("dropout_0", False)

project_name = "multi-layers"
project_directory = "cnn_search"

tuner = HistoryDeletingBayesianOptimization(
    TunableModelFamilyHypermodel((3, 224, 224), TunableConvNetFamily()),
    hyperparameters=hyperparameters,
    objective='val_loss',
    tune_new_entries=True,
    executions_per_trial=1,
    overwrite=False,
    directory=project_directory,
    max_trials=15,
    project_name=project_name
)

Reloading Tuner from cnn_search/multi-layers/tuner0.json


In [6]:
search_has_been_done = True  # To avoid overriding my stuff.
if not search_has_been_done:
    tuner.search(train_dataloader, epochs=12, validation_data=validation_dataloader,
                 callbacks=[keras.callbacks.CSVLogger(
                     f"{project_directory}/{project_name}/search.log", separator=",", append=True)
                 ])

Trial 1  was skipper for unknown reasons

In [7]:
tuner.results_summary(5)

Results summary
Results in cnn_search/multi-layers
Showing 5 best trials
Objective(name="val_loss", direction="min")

Trial 08 summary
Hyperparameters:
lr: 0.0001
momentum: 0.9
hidden_layers: 1
dropout_0: False
convolution_layers: 2
filters_0: 128
kernel_0: 5
units_0: 32
filters_1: 64
kernel_1: 3
Score: 0.4289422035217285

Trial 04 summary
Hyperparameters:
lr: 0.0001
momentum: 0.9
hidden_layers: 1
dropout_0: False
convolution_layers: 2
filters_0: 16
kernel_0: 5
units_0: 64
filters_1: 128
kernel_1: 5
Score: 0.43078967928886414

Trial 12 summary
Hyperparameters:
lr: 0.0001
momentum: 0.9
hidden_layers: 1
dropout_0: False
convolution_layers: 2
filters_0: 128
kernel_0: 3
units_0: 64
filters_1: 128
kernel_1: 5
Score: 0.4354296922683716

Trial 02 summary
Hyperparameters:
lr: 0.0001
momentum: 0.9
hidden_layers: 1
dropout_0: False
convolution_layers: 1
filters_0: 32
kernel_0: 5
units_0: 64
filters_1: 64
kernel_1: 5
Score: 0.4389541447162628

Trial 10 summary
Hyperparameters:
lr: 0.0001
momentum

In [19]:
import pandas

csv = pandas.read_csv("./cnn_search/multi-layers/search.log")
csv['tuner_iteration'] = 0

current_iteration = 0
for index, row in enumerate(csv.itertuples()):
    csv.at[index, 'tuner_iteration'] = int(index / 12) + 1

best_dataframe = csv.query("tuner_iteration in [8, 4, 12, 2, 10]")

In [25]:
import plotly.express as px

loss_figure = px.line(best_dataframe, x="epoch", y=["loss"], color="tuner_iteration", template="plotly_white",
                      markers=True)
loss_figure.update_layout(title="Loss in tuner search", xaxis_title="Epoch", yaxis_title="Loss")

In [24]:
figure = px.line(best_dataframe, x="epoch", y=["val_loss"], color="tuner_iteration", template="plotly_white",
                 markers=True)
figure.update_layout(title="Validation Loss in tuner search", xaxis_title="Epoch", yaxis_title="Loss")

In [26]:
# 4 Looks promising and also 8 is good. We keep those two

In [27]:
best_hyperparameters_references = [
    dict(iteration=8, hyperparameters_index=0),
    dict(iteration=4, hyperparameters_index=1),
]

[print(f"iteration:{i['iteration']}, hp: {tuner.get_best_hyperparameters(5)[i['hyperparameters_index']].values}") for i
 in best_hyperparameters_references]

iteration:8, hp: {'lr': 0.0001, 'momentum': 0.9, 'hidden_layers': 1, 'dropout_0': False, 'convolution_layers': 2, 'filters_0': 128, 'kernel_0': 5, 'units_0': 32, 'filters_1': 64, 'kernel_1': 3}
iteration:4, hp: {'lr': 0.0001, 'momentum': 0.9, 'hidden_layers': 1, 'dropout_0': False, 'convolution_layers': 2, 'filters_0': 16, 'kernel_0': 5, 'units_0': 64, 'filters_1': 128, 'kernel_1': 5}


[None, None]